# MO&PC Project

In [1]:
import pandas as pd
import sqlite3
import os

notebook_folder = os.getcwd()

pagos_unificados_path = os.path.join(notebook_folder, 'pagos_unificados.csv')
periodos_tableros_path = os.path.join(notebook_folder, 'periodos_tableros.csv')

# 1. Creación de una DB SQL cloud o local:
database_path = os.path.join(notebook_folder, 'PROJECTMOPCDATABASE.db')

pagos_unificados = pd.read_csv(pagos_unificados_path)
periodos_tableros = pd.read_csv(periodos_tableros_path)

conn = sqlite3.connect(database_path)

pagos_unificados.to_sql('PAGOS_UNIFICADOS', conn, index=False, if_exists='replace')
periodos_tableros.to_sql('PERIODOS_TABLEROS', conn, index=False, if_exists='replace')

# 2. Consulta SQL para luego generar una vista que contenga la información necesaria para la visualización:

query = """
SELECT 
    CASE 
        WHEN SUBSTR(pt.IDCLIENTE, 1, 4) = 'AMEX' THEN 'AMEX'
        WHEN SUBSTR(pt.IDCLIENTE, 1, 4) = 'HSBC' THEN 'HSBC'
        ELSE 'OTROS'
    END AS Categoria,
    pt.IDCLIENTE,
    pt.Periodo,
    pt.Finicio AS Fecha_Inicio,
    COALESCE(SUM(pu.Monto), 0) AS Suma_Monto,
    CASE 
        WHEN pt.Periodo = 'julio2021' THEN '2021-07-01'
        WHEN pt.Periodo = 'junio2021' THEN '2021-06-01'
        WHEN pt.Periodo = 'mayo2021' THEN '2021-05-01'
        WHEN pt.Periodo = 'abril2021' THEN '2021-04-01'
        WHEN pt.Periodo = 'marzo2021' THEN '2021-03-01'
        WHEN pt.Periodo = 'febrero2021' THEN '2021-02-01'
        ELSE NULL
    END AS ComienzoDeMesCalendario
FROM 
    PERIODOS_TABLEROS pt,
    PAGOS_UNIFICADOS pu
WHERE
    pu.IDCLIENTE = pt.IDCLIENTE
    AND (
        (pu.Fuente_Pago = 'MEMO_PAGO_OPERADOR' AND pu.Fecha_remesa IS NOT NULL AND pu.Fecha_remesa BETWEEN pt.Finicio AND pt.Ffin)
        OR
        (pu.Fuente_Pago = 'MEMO_PAGO_OPERADOR' AND pu.Fecha_remesa IS NULL AND pu.Fecha_pago BETWEEN pt.Finicio AND pt.Ffin)
        OR
        (pu.Fuente_Pago <> 'MEMO_PAGO_OPERADOR' AND pu.Fecha_pago BETWEEN pt.Finicio AND pt.Ffin)
    )
    AND pt.Periodo IN (
        SELECT Periodo
        FROM (
            SELECT 
                pt.IDCLIENTE,
                pt.Periodo,
                ROW_NUMBER() OVER (PARTITION BY pt.IDCLIENTE ORDER BY pt.Finicio DESC) AS rn
            FROM 
                PERIODOS_TABLEROS pt
        ) AS Subquery
        WHERE rn <= 6
    )
GROUP BY 
    Categoria,
    pt.IDCLIENTE,
    pt.Periodo,
    pt.Finicio
ORDER BY 
    Categoria, 
    pt.IDCLIENTE, 
    pt.Finicio DESC;
"""

# Creación de la vista utilizando la consulta anterior, una vez chequeado su funcionamiento
conn.execute("CREATE VIEW IF NOT EXISTS ViewSeisUltimosPeriodosMOPC AS " + query)

# Acceso a la vista
result = pd.read_sql_query("SELECT * FROM ViewSeisUltimosPeriodosMOPC;", conn)

pd.set_option('display.max_rows', None)
print(result)

# Lee la base de datos y accede a la vista generada
result = pd.read_sql_query("SELECT * FROM ViewSeisUltimosPeriodosMOPC;", conn)

# 3. y 4. Materializar la vista en una tabla utilizando Python:
result.to_csv('ViewSeisUltimosPeriodosMOPC.csv', index=False) 

# 5. Dashboard utilizando el spreadsheet generado:   https://lookerstudio.google.com/reporting/aeb7aa24-88ec-40c7-a77f-2df4c01f7980


   Categoria          IDCLIENTE      PERIODO Fecha_Inicio  Suma_Monto  \
0       AMEX          AMEX_ALHR    julio2021   2021-06-29    27900.00   
1       AMEX          AMEX_ALHR    junio2021   2021-05-29    27126.00   
2       AMEX          AMEX_ALHR     mayo2021   2021-04-28    39310.42   
3       AMEX          AMEX_ALHR    abril2021   2021-03-30    23000.00   
4       AMEX          AMEX_ALHR    marzo2021   2021-02-26   281930.07   
5       AMEX          AMEX_ALHR  febrero2021   2021-01-28    38684.00   
6       AMEX          AMEX_ALLR    julio2021   2021-06-29     8185.00   
7       AMEX          AMEX_ALLR    junio2021   2021-05-29    16432.44   
8       AMEX          AMEX_ALLR     mayo2021   2021-04-28     9000.00   
9       AMEX          AMEX_ALLR    abril2021   2021-03-30    33529.33   
10      AMEX          AMEX_ALLR    marzo2021   2021-02-26     4700.00   
11      AMEX          AMEX_ALLR  febrero2021   2021-01-28     6845.25   
12      AMEX          AMEX_PALM    julio2021   2021